# Introduction to Standard Conventions

A `convention` is simply a set of standard attributes. The standard attributes are defined in a YAML file, which can be loaded by the `h5RDMtoolbox`. Then, it can be registered for the session or now or later use. 

In [1]:
import h5rdmtoolbox as h5tbx
from h5rdmtoolbox import tutorial

## Import/Load a convention

A convention can be read in **from a local directory**. It then is stored in a YAML file:

In [2]:
cv = h5tbx.conventions.from_yaml(tutorial.get_standard_attribute_yaml_filename())

The other option is to load one **from an online resource**. The toolbox supports [Zenodo](https://zenodo.org/):

In [3]:
# cv = h5tbx.conventions.from_zenodo(doi=8220739)

## Enable (use) a convention

To make a convention affective, it must be enabled. We do this by calling either the method `use()` of the convention ...

In [9]:
cv.use()
print(h5tbx.conventions.get_current_convention().name)

planar-piv-convention


... or we could also call the package function `use()`:

In [10]:
h5tbx.use('h5tbx')

Note, the above "h5tbx" convention is available by default, thus is already registered.

We can also **disable** the current convention by simply passing `None`:

In [11]:
h5tbx.use(None)

A convention can also only be **temporarily** be enabled by using the context-manager-syntax. Here are two examples:

In [15]:
with h5tbx.use(None):
    # create a temporary file
    with h5tbx.File() as h5:
        h5.create_dataset('non_regulated_dataset', data=4)

with h5tbx.use('h5tbx'):  # this concention requires the attribute "units" for datasets:
    # create a temporary file
    with h5tbx.File() as h5:
        h5.create_dataset('non_regulated_dataset', data=4, units='m/s')